In [1]:
# import everything that I will use
import re
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from datetime import date
import datetime
import requests
from tqdm.auto import tqdm




/Users/luizfernandotoledo/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
#open Selenium Chrome webdriver
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/96.0.4664.45/chromedriver_mac64.zip
Driver has been saved in cache [/Users/luizfernandotoledo/.wdm/drivers/chromedriver/mac64/96.0.4664.45]
/var/folders/dj/5c61qy0d0b9gcj0yrt1jd2300000gn/T/ipykernel_50410/1903745058.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
#find a link that gives me the last 30 days FOIA requests with the keyword 'Bolsonaro'
# if you want to change the keyword, just change the variable search_words

# creating a link
keyword = 'bolsonaro'
today = date.today()
days = datetime.timedelta(30)
thirty_days_ago = today - days
thirty_days_ago = str(thirty_days_ago)
year_regex = r"(\d\d\d\d)-"
year_only = re.findall(year_regex, thirty_days_ago)[0]
month_regex = r".....(..)"
month_only = re.findall(month_regex, thirty_days_ago)[0]
day_regex = r"........(..)"
day_only = re.findall(day_regex, thirty_days_ago)[0]


### gambiarra monstra pra pegar a URL pq ainda to aprendendo, rs
useful_link1 = 'http://www.consultaesic.cgu.gov.br/busca/SitePages/resultadopesquisa.aspx?k=ALL('
useful_link2 = keyword
useful_link3 = ')%20DataPedido%3E%3D'
useful_link4 = day_only
useful_link5 = '%2F'
useful_link6 = month_only
useful_link7 = '%2F'
useful_link8 = year_only

useful_link = useful_link1 + useful_link2 + useful_link3 + useful_link4 + useful_link5 + useful_link6 + useful_link7 + useful_link8  


In [4]:
#driver opens the link needed in order to find all FOIA requests

driver.get(useful_link)
 

In [5]:
#step one: create a list with links from all the pages


list_of_links = []
#try to open up to 100 pages in order to find those links
for i in tqdm(range(100)):
    for pedido in driver.find_elements(By.CLASS_NAME, 'ms-srch-ellipsis'):
        links = pedido.find_elements(By.TAG_NAME, 'a')
        for link in links:
            caso = link.get_attribute('href')
            list_of_links.append(caso)
    #click on the next page
    try:
        driver.find_element(By.XPATH, '//*[@id="PageLinkNext"]').click()
        time.sleep(2)
    except:
        break


  0%|          | 0/100 [00:00<?, ?it/s]

In [6]:
#check if we've got everything
list_of_links
# count_content = len(list_of_contents)

# print(f'There are {count_link} links and {count_content} contents.')

['http://www.consultaesic.cgu.gov.br/busca/dados/Lists/Pedido/DispForm.aspx?ID=1595343',
 'http://www.consultaesic.cgu.gov.br/busca/dados/Lists/Pedido/DispForm.aspx?ID=1595209',
 'http://www.consultaesic.cgu.gov.br/busca/dados/Lists/Pedido/DispForm.aspx?ID=1595340',
 'http://www.consultaesic.cgu.gov.br/busca/dados/Lists/Pedido/DispForm.aspx?ID=1595282',
 'http://www.consultaesic.cgu.gov.br/busca/dados/Lists/Pedido/DispForm.aspx?ID=1591834',
 'http://www.consultaesic.cgu.gov.br/busca/dados/Lists/Pedido/DispForm.aspx?ID=1594134',
 'http://www.consultaesic.cgu.gov.br/busca/dados/Lists/Pedido/DispForm.aspx?ID=1594907',
 'http://www.consultaesic.cgu.gov.br/busca/dados/Lists/Pedido/DispForm.aspx?ID=1592942',
 'http://www.consultaesic.cgu.gov.br/busca/dados/Lists/Pedido/DispForm.aspx?ID=1592742',
 'http://www.consultaesic.cgu.gov.br/busca/dados/Lists/Pedido/DispForm.aspx?ID=1594376',
 'http://www.consultaesic.cgu.gov.br/busca/dados/Lists/Pedido/DispForm.aspx?ID=1592345']

In [7]:
# create a list and fill it with request title, request link and request text


lista_de_pedidos = []
for link in tqdm(list_of_links):
    lista_do_pedido = []
    driver.get(link)
    resumao = driver.find_element(By.XPATH, '/html/body/form/div[4]/div[10]/div[1]/div[2]/div/div[2]/div[2]/div[1]/table/tbody/tr/td/div/div[3]/div[1]/div/div[1]/span/div[2]/div/div[2]/div/div/div/div[2]/div[1]/div/div/fieldset/div/div[1]/span/textarea')
    lista_do_pedido.append(resumao.text)
    link_do_pedido = driver.find_element(By.XPATH, '/html/body/form/div[4]/div[10]/div[1]/div[2]/div/div[2]/div[2]/div[1]/table/tbody/tr/td/div/div[3]/div[1]/div/div[1]/span/div[2]/div/div[2]/div/div/div/div[2]/div[2]/div/div/fieldset/div/div[3]/table/tbody/tr/td[2]/div/span/textarea')
    lista_do_pedido.append(link_do_pedido.text)
    texto_do_pedido = driver.find_element(By.XPATH, '/html/body/form/div[4]/div[10]/div[1]/div[2]/div/div[2]/div[2]/div[1]/table/tbody/tr/td/div/div[3]/div[1]/div/div[1]/span/div[2]/div/div[2]/div/div/div/div[2]/div[4]/div/div/fieldset/div/div/table/tbody/tr/td[2]/div[1]/span/div')
    lista_do_pedido.append(texto_do_pedido.text)
    lista_de_pedidos.append(lista_do_pedido)

lista_de_pedidos


  0%|          | 0/11 [00:00<?, ?it/s]

[['Motociata Bolsonaro - Pedido 08198036667202125',
  'http://www.consultaesic.cgu.gov.br/busca/_layouts/15/DetalhePedido/DetalhePedido.aspx?nup=08198036667202125',
  'Com base no artigo 5º (XXXIII) da Constituição Federal e nos artigos 10, 11 e 12 da Lei nº 12.527/2011, a Lei Geral de Acesso a Informações Públicas, solicito a seguinte informação: valor total dos gastos com diárias, alimentação, combustível, e qualquer outro recurso necessário para reforçar o policiamento da motociata do presidente Jair Bolsonaro, no dia 9 de maio, em Brasília, assim como o número do efetivo reforçado.'],
 ['Transmissão eventos EBC - Pedido 53125001689202182',
  'http://www.consultaesic.cgu.gov.br/busca/_layouts/15/DetalhePedido/DetalhePedido.aspx?nup=53125001689202182',
  'Bom dia, visto que nos últimos tempos a EBC tem passado por inúmeras mudanças, inclusive com a extinção da NBR e incorporação de algumas de suas características pela Tv Brasil venho através desse requerimento pedir esclarecimento ac

In [8]:
import pandas

df = pd.DataFrame(lista_de_pedidos, columns = ['resumo', 'link', 'texto_do_pedido'])

df.to_csv('lista_de_pedidos.csv')

